# 1. Load Dataset

In [1]:
import numpy as np
%env HF_HOME=/media/automan/ExSpace/PreData

env: HF_HOME=/media/automan/ExSpace/PreData


# 3.Load Model

In [2]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, AutoTokenizer
from qwen_vl_utils import process_vision_info
from model.VideoLISA import VideoLISA

model_path = "Qwen/Qwen2.5-VL-3B-Instruct"

model = VideoLISA.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)
model.init_sam_module(model_path="/media/automan/ExSpace/Projects/VideoLISA/checkpoints/sam_vit_h_4b8939.pth")
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
tokenizer.add_tokens("<seg>", special_tokens=False)
model.seg_token_idx = tokenizer.convert_tokens_to_ids("<seg>")
model.resize_token_embeddings(len(tokenizer))
processor = AutoProcessor.from_pretrained(model_path)
processor.tokenizer = tokenizer
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/media/automan/ExSpace/Projects/VideoLISA/videolisa/model/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)
Using a

In [3]:
for param in model.parameters():
    param.requires_grad = False
print(model.lm_head.weight.requires_grad)
print(model.base_model.embed_tokens.weight.requires_grad)
model.lm_head.weight.requires_grad = True
model.base_model.embed_tokens.weight.requires_grad = True
print(model.lm_head.weight.requires_grad)
print(model.base_model.embed_tokens.weight.requires_grad)

False
False
True
True


In [4]:
print(model.seg_token_idx)
print(tokenizer("<seg>"))
# tokenizer.decode([151665])
embedding_size = model.get_input_embeddings().weight.shape
lm_head_size = model.lm_head.weight.shape
pre_lm_weight = model.lm_head.weight.clone()

151665
{'input_ids': [151665], 'attention_mask': [1]}


In [5]:
import torch.nn as nn

# 获取模型的输入嵌入层
embedding_layer = model.get_input_embeddings()

# 选择参考token：使用start_header_id token
reference_token_id = tokenizer.convert_tokens_to_ids("<|file_sep|>")

# 将参考token的嵌入权重复制到新token
for token in ["<seg>"]:
    token_id = tokenizer.convert_tokens_to_ids(token)
    embedding_layer.weight.data[token_id] = embedding_layer.weight.data[reference_token_id].clone()

# 4.Train Model

In [3]:
import torch
from datasets import Dataset
from qwen_vl_utils import process_vision_info
from peft import LoraConfig, TaskType, get_peft_model, PeftModel
from transformers import (
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
)
import json
from utils import SHORT_QUESTION_LIST, ANSWER_LIST
import random
import numpy as np
import cv2

def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 8192
    input_ids, attention_mask, labels = [], [], []
    conversation = example["conversations"]
    input_content = conversation[0]["value"]
    # output_content = conversation[1]["value"]
    output_content = random.choice(ANSWER_LIST)
    file_path = input_content.split("<|vision_start|>")[1].split("<|vision_end|>")[0]  # 获取图像路径
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": f"{file_path}",
                    "resized_height": 280,
                    "resized_width": 280,
                },
                {"type": "text", "text": f"{random.choice(SHORT_QUESTION_LIST)}"},
            ],
        }
    ]
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )  # 获取文本
    image_inputs, video_inputs = process_vision_info(messages)  # 获取数据数据（预处理过）
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = {key: value.tolist() for key, value in inputs.items()} #tensor -> list,为了方便拼接
    instruction = inputs

    response = tokenizer(f"{output_content}", add_special_tokens=False)

    input_ids = (
            instruction["input_ids"][0] + response["input_ids"] + [tokenizer.pad_token_id]
    )

    attention_mask = instruction["attention_mask"][0] + response["attention_mask"] + [1]
    labels = (
            [-100] * len(instruction["input_ids"][0])
            + response["input_ids"]
            + [tokenizer.pad_token_id]
    )
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    labels = torch.tensor(labels)
    inputs['pixel_values'] = torch.tensor(inputs['pixel_values'])
    inputs['image_grid_thw'] = torch.tensor(inputs['image_grid_thw']).squeeze(0)  #由（1,h,w)变换为（h,w）
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels,
            "pixel_values": inputs['pixel_values'], "image_grid_thw": inputs['image_grid_thw']}

def predict(messages, model):
    # 准备推理
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # 生成输出
    generated_ids, pred_masks = model.generate(original_images=image_inputs, **inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    # output_text = processor.batch_decode(
    #     generated_ids_trimmed, skip_special_tokens=False, clean_up_tokenization_spaces=False
    # )
    image_np = np.array(image_inputs[0])
    pred_mask = pred_masks[0][0].to(bool).cpu().numpy()
    highlight = np.zeros_like(image_np, dtype=np.uint8)
    highlight[pred_mask] = (255, 0, 0)
    # 将高亮遮罩与原图叠加
    highlighted_image = cv2.addWeighted(image_np, 0.5, highlight, 0.5, 0)
    return output_text[0], highlighted_image


In [4]:
# 处理数据集：读取json文件
# 拆分成训练集和测试集，保存为data_vl_train.json和data_vl_test.json
train_json_path = "data_vl.json"
with open(train_json_path, 'r') as f:
    data = json.load(f)
    train_data = data[:-4]
    test_data = data[-4:]

with open("data_vl_train.json", "w") as f:
    json.dump(train_data, f)

with open("data_vl_test.json", "w") as f:
    json.dump(test_data, f)

train_ds = Dataset.from_json("data_vl_train.json")
train_dataset = train_ds.map(process_func)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

In [5]:
# 配置LoRA
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,  # 训练模式
    r=64,  # Lora 秩
    lora_alpha=16,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.05,  # Dropout 比例
    bias="none",
)

# 获取LoRA模型
peft_model = get_peft_model(model, config)

In [9]:
peft_model.base_model.lm_head.weight.requires_grad = True
peft_model.base_model.model.model.embed_tokens.weight.requires_grad = True

In [10]:
print(peft_model.base_model.lm_head.weight.requires_grad)
print(peft_model.base_model.model.model.embed_tokens.weight.requires_grad)

True
True


In [6]:
# 配置训练参数
args = TrainingArguments(
    output_dir="./output/Qwen2_5-VL-3B",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=10,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

# 配置Trainer
trainer = Trainer(
    model=peft_model,
    # model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)
# 开启模型训练
trainer.train()

[2025-03-30 18:06:01,801] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/automan/Softwares/miniconda3/envs/videolisa/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/automan/Softwares/miniconda3/envs/videolisa/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/automan/Softwares/miniconda3/envs/videolisa/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/automan/Softwares/miniconda3/envs/videolisa/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/automan/Softwares/miniconda3/envs/videolisa/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/automan/Softwares/miniconda3/envs/videolisa/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undef

TypeError: VideoLISA.forward() missing 2 required positional arguments: 'original_images' and 'gt_masks'

In [ ]:
post_lm_weight = peft_model.lm_head.weight.clone()
sum(sum(pre_lm_weight - post_lm_weight))

In [12]:
post_lm_weight = model.lm_head.weight.clone()
sum(sum(pre_lm_weight - post_lm_weight))
# print(tokenizer("<|endoftext|>"))

{'input_ids': [151643], 'attention_mask': [1]}


In [14]:
# 读取测试数据
with open("data_vl_test.json", "r") as f:
    test_dataset = json.load(f)
test_image_list = []
for item in test_dataset:
    input_image_prompt = item["conversations"][0]["value"]
    # 去掉前后的<|vision_start|>和<|vision_end|>
    origin_image_path = input_image_prompt.split("<|vision_start|>")[1].split("<|vision_end|>")[0]

    messages = [{
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": origin_image_path
            },
            {
                "type": "text",
                "text": f"{random.choice(SHORT_QUESTION_LIST)}"
            }
        ]}]

    response = predict(messages, model)
    messages.append({"role": "assistant", "content": f"{response}"})
    print(messages[-1])

/home/automan/Softwares/miniconda3/envs/videolisa/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


AttributeError: 'Tensor' object has no attribute 'hidden_states'

# 5.Test Model

In [4]:
# ===测试模式===
# 配置测试参数
val_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=True,
    r=8,  # Lora 秩
    lora_alpha=16,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.05,  # Dropout 比例
    bias="none",
)
# 获取测试模型
val_peft_model = PeftModel.from_pretrained(model, model_id="/media/automan/ExSpace/Projects/VideoLISA/output/VideoLISA_lora-8_frame-16/checkpoint-600", config=val_config)


In [5]:
# 读取测试数据
with open("data_vl_test.json", "r") as f:
    test_dataset = json.load(f)
test_image_list = []
for item in test_dataset:
    # input_image_prompt = item["conversations"][0]["value"]
    # # 去掉前后的<|vision_start|>和<|vision_end|>
    # origin_image_path = input_image_prompt.split("<|vision_start|>")[1].split("<|vision_end|>")[0]

    messages = [{
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": origin_image_path
            },
            {
                "type": "text",
                "text": f"{random.choice(SHORT_QUESTION_LIST)}"
            }
        ]}]


    messages.append({"role": "assistant", "content": f"{response}"})
    print(messages[-1])

NameError: name 'origin_image_path' is not defined

In [7]:
origin_image_path = "/media/automan/6E94666294662CB1/A_Content/Datasets/ADEChallengeData2016/images/train/ADE_train_00000001.jpg"
messages = [{
    "role": "user",
    "content": [
        {
            "type": "image",
            "image": origin_image_path
        },
        {
            "type": "text",
            "text": "Can you segment the floor in this image?"
        }
    ]}]

response, image = predict(messages, val_peft_model)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.imwrite("../output/image.png", image.astype(np.uint8))
messages.append({"role": "assistant", "content": f"{response}"})
print(messages[-1])

{'role': 'assistant', 'content': 'Sure, it is <seg>.'}


In [ ]:
final_lm_weight = val_peft_model.lm_head.weight.clone()
sum(sum(pre_lm_weight - final_lm_weight))

In [5]:
mw = model.lm_head.weight
print(mw)

Parameter containing:
tensor([[ 0.0325,  0.0078,  0.0029,  ..., -0.0220,  0.0128,  0.0175],
        [ 0.0093,  0.0025,  0.0004,  ..., -0.0089, -0.0142,  0.0240],
        [-0.0383, -0.0148,  0.0097,  ..., -0.0208,  0.0044,  0.0258],
        ...,
        [-0.0229,  0.0014, -0.0175,  ...,  0.0154,  0.0069,  0.0018],
        [-0.0229,  0.0014, -0.0175,  ...,  0.0154,  0.0069,  0.0018],
        [-0.0229,  0.0014, -0.0175,  ...,  0.0154,  0.0069,  0.0018]],
       device='cuda:0', dtype=torch.bfloat16, requires_grad=True)


In [7]:
vw = val_peft_model.base_model.model.lm_head.weight
print(vw)

Parameter containing:
tensor([[ 0.0325,  0.0078,  0.0029,  ..., -0.0220,  0.0128,  0.0175],
        [ 0.0093,  0.0025,  0.0004,  ..., -0.0089, -0.0142,  0.0240],
        [-0.0383, -0.0148,  0.0097,  ..., -0.0208,  0.0044,  0.0258],
        ...,
        [-0.0229,  0.0014, -0.0175,  ...,  0.0154,  0.0069,  0.0018],
        [-0.0229,  0.0014, -0.0175,  ...,  0.0154,  0.0069,  0.0018],
        [-0.0229,  0.0014, -0.0175,  ...,  0.0154,  0.0069,  0.0018]],
       device='cuda:0', dtype=torch.bfloat16)


In [8]:
sum(sum(mw - vw))

tensor(0., device='cuda:0', dtype=torch.bfloat16)

In [9]:
from safetensors.torch import load_file
d = load_file("/media/automan/ExSpace/Projects/VideoLISA/videolisa/output/Qwen2_5-VL-3B/checkpoint-150/adapter_model.safetensors")
sw = d["base_model.model.lm_head.weight"].data

In [11]:
sum(sum(sw.to("cuda") - vw))

tensor(0., device='cuda:0', dtype=torch.bfloat16)

In [30]:
a = tokenizer.decode(199)
print(a)